# Schema Matching & Entity Linking
#### Step 1) Loading our extracted data!
Here, we are repeating the last step of the first tutorial to create a data-frame containing the features that we are interested in.

In [1]:
import pandas as pd
import numpy as np
import spacy
import re

nlp = spacy.load('en')

incidents = []
with open('./data/aviation_incidents_cleaned.txt') as myfile:
    for line in myfile:
        doc = nlp(line)
        icd = {'Date':'', 'Airline':'', 'Flight':'', 'Location':'', 'Text':''}
        for ent in doc.ents:
            if ent.text.strip() == '': continue     # skip empty strings
            if ent.label_ == u'DATE':               # Get the date
                icd['Date'] = ent.text
            elif ent.label_ == u'ORG':              # Get the organization
                icd['Airline'] = ent.text                
            elif ent.label_ == u'PRODUCT':          # Get the flight number
                if re.search(r'flight', ent.text, re.I):
                    icd['Flight'] = ent.text
            elif ent.label_ == u'GPE':              # Get the location
                if icd['Location'] == '':
                    icd['Location'] = ent.text
                else:
                    icd['Location'] += ', ' + ent.text
        icd['Text'] = line
        incidents.append(icd)

df = pd.DataFrame(incidents)
df.sample(3)

,Airline,Date,Flight,Location,Text
235,,"November 23, 1961",Aerolíneas Argentinas Flight,"a de Havilland Comet, Campinas, Brazil","November 23, 1961, Aerolíneas Argentinas Fligh..."
886,Luxair,"November 6, 2002",Flight 9642,Fokker F50,"November 6, 2002, Luxair Flight 9642, a Fokker..."
909,Boeing,"February 3, 2005",,Afghanistan,"February 3, 2005, Kam Air Flight 904, a Boeing..."


#### Step 2) Reading a new data source
Assume that we have came across a new dataset on flight indicents, and we would like to combine this dataset with what we extracted earlier. The first step would be to read the data.

In [2]:
external_df = pd.read_csv('data/crashes.csv', dtype='str')
external_df.sample(3)

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
2686,09/30/1973,20:40,"Near Sverdlovsk, Russia",Aeroflot,NaN,NaN,Tupolev TU-104B,CCCP-42506,21904,108,108,0,The aircraft crashed shortly after takeoff whi...
2690,10/29/1973,NaN,Sainte Lucia Island,CATA,NaN,NaN,Britten-Norman BN-2A-6 Islander,N37JA,210,4,4,0,Flew into Mt. Gimie while en route.
4556,04/20/1998,16:47,"Near Bogota, Colombia",Air France,422,Bogota - Quito,Boeing B-727-230,HC-BSU,21622/1431,53,53,0,"The aircraft, leased from TAME, crashed atop f..."


---
## Schema Matching

#### Step 3) Figuring out how the two datasets are related
We need to figure out which columns in the new dataset corresponds to the columns in our dataset. Obviously humans can do this manually by reviewing the names assigned to each column as well as looking at the values listed under each column. However, there are many cases in which this task becomes exteremely cumbersome (e.g., when there are many data sources, or when data-sources are found with no proper description). In this part, we demonstrate some technique to automatically find which columns are related to each other.

Let's start by creating subset of both datasets where there are no missing values.

In [3]:
full_df = df.replace(r'^\s*$', np.nan, regex=True).dropna(axis=0)
full_external_df = external_df.replace(r'^\s*$', np.nan, regex=True).dropna(axis=0)

In [4]:
print(len(full_df))
print(len(full_external_df))

422
944


#### Step 4) Using Jaccard similarity to match the columns
It is safe to say that columns that share similar values are of the same type. To measure the similarity of the values that appear in two columns, we can use the jaccard similarity. Let's start by computing the similarity of all columns in the external data with the column 'Airline' in our dataset.

In [5]:
col_vals = set(full_df['Airline'].unique())
for col in full_external_df.columns:
    ext_col_vals = set(full_external_df[col].unique())
    intersection_size = len(col_vals.intersection(ext_col_vals))
    union_size = len(col_vals.union(ext_col_vals))
    jaccard = intersection_size / union_size
    if jaccard != 0:
        print(col + ' has similarity score ' + str(jaccard))

Operator has similarity score 0.04017857142857143
Type has similarity score 0.006361323155216285


Now let's try the same technique for column 'Date'.

In [6]:
col_vals = set(full_df['Date'].unique())
for col in full_external_df.columns:
    ext_col_vals = set(full_external_df[col].unique())
    intersection_size = len(col_vals.intersection(ext_col_vals))
    union_size = len(col_vals.union(ext_col_vals))
    jaccard = intersection_size / union_size
    if jaccard != 0:
        print(col + ' has similarity score ' + str(jaccard))

Flight # has similarity score 0.0037209302325581397
cn/In has similarity score 0.0022488755622188904
Aboard has similarity score 0.0015923566878980893
Fatalities has similarity score 0.001694915254237288


We can see that the jaccard similarity works well when we deal with categorical features which are limited to a fixed set of values. However, for attributes such as 'Date', we can see that jaccard similarity is not performing well. This is because Dates while having similar format can have many distinct values. 

#### Step 5) Using FlexMatcher to match the schemas
FlexMatcher is package that uses maching learning technqiues to figure out which columns should be mapped together. For instance, while the extact values under column 'Date' may not be identical, FlexMatcher can detect the common patterns and map the column to another column that exhibits similar patterns.

FlexMatcher works as follows. It takes as input a list of dataframes and list of dictionaries that describe how the columns in the input dataframes map to the columns that we are interested in. Once this data is provided, FlexMatcher learns the common patterns. Then, we provide a new dataframe and the FlexMatcher would predict which columns of the new dataset are related to the columns we are interested in. 

Since, we only have two dataframes, we need to simplify our setting. We use our extracted datset to both train on, and the columns we are interested in are exactly the 5 columns that are in the extracted datset. Then, we ask FlexMatcher to make a prediction about the external datasource.

In [7]:
import flexmatcher

# training FlexMatcher
schema_list = [full_df]
mapping_list = [dict(zip(full_df.columns, full_df.columns))]
fm = flexmatcher.FlexMatcher()
fm.train(schema_list, mapping_list, sample_size=500)

Setting up FlexMatcher ...
Creating Tranining Data ...


In [8]:
mapping_list[0]

{'Airline': 'Airline',
 'Date': 'Date',
 'Flight': 'Flight',
 'Location': 'Location',
 'Text': 'Text'}

In [9]:
# making a prediction
predicted_mapping = fm.predict(full_external_df, predict_all=False)
for t, v in predicted_mapping.items():
    print(t, '====', v)

Date ==== Flight
Location ==== Location
Operator ==== Airline
cn/In ==== Date
Summary ==== Text


/Users/behzad/miniconda2/envs/py3flex/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


It's important to remember that machine learning techniques also make mistakes. In this case, FlexMatcher fails to make a correct prediction for the attribute 'Flight' which should clearly be mapped to column 'Flight #'. It's easy to see why FlexMatcher is confused. The values under 'Flight' in mostly contain the word "Flight" in them:

In [10]:
full_df['Flight'].sample(3)

428              Flight 440
1091         AirAsia Flight
511     Air Rhodesia Flight
Name: Flight, dtype: object

While the external datset only list numbers:

In [11]:
full_external_df['Flight #'].sample(3)

4789     352
4040    2574
2694     160
Name: Flight #, dtype: object

This makes it for the learning algorithm to see the similarities. In fact, the algorithm sees more similarities between the 'Flight #' column and other column such as 'Aboard' which represents the number of people on the flight

---
## Entity Linking

Now that we know how the columns of the two datasets are related, we can try to merge the two datasets. Our goal is to augment our dataset which additional features that we can find in the external data source. We assume two datapoints refer to the same incident if the listed Airline and the flight number are identical. 

#### Step 6) Finding matching attributes using dataframes
Recall that there is a mistmatch between the format of the flight numbers. We start by cleaning the flight number in our dataset to only keep the numbers (and drop the word flight). Then we can use dataframes to find matching incidents.

In [12]:
import re
clean_df = full_df.copy()
clean_df['Flight'] = clean_df['Flight'].apply(lambda x: '-'.join(re.findall(r'\d+', x)))

In [13]:
combined = clean_df.merge(full_external_df,
                         right_on=['Operator', 'Flight #'],
                         left_on=['Airline', 'Flight'], how='inner')
print('We have found ' + str(len(combined)) + ' matching records!')
combined.head(3)

We have found 32 matching records!


,Airline,Date_x,Flight,Location_x,Text,Date_y,Time,Location_y,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,American Airlines,"January 14, 1936",1,"Goodwin, Arkansas","January 14, 1936, American Airlines Flight 1, ...",01/14/1936,19:32,"Goodwin, Arkansas",American Airlines,1,"Newark, NJ - Fort Worth, TX",Douglas DC-2-120,NC14274,1307,17,17,0,Flew into trees and disintegrated. The cause o...
1,American Airlines,"January 14, 1936",1,"Goodwin, Arkansas","January 14, 1936, American Airlines Flight 1, ...",10/30/1941,22:10,"St. Thomas, Ontario, Canada",American Airlines,1,New York - Buffalo - Chicago - Detroit,Douglas DC-3,NC25663,2207,20,20,0,"The aircraft, on a flight from New York to Chi..."
2,American Airlines,"January 14, 1936",1,"Goodwin, Arkansas","January 14, 1936, American Airlines Flight 1, ...",03/01/1962,10:09,"Jamaica Bay, New York, New York",American Airlines,1,New York City - Los Angeles,Boeing B-707-123B,N7506A,17633/12,95,95,0,After taking off from Idlewild Airport and rea...


This is interesting, but is that all?

#### Step 7) Trying approximate string matching using the py_stringsimjoin package
The main problem with an exact match is that there might be slight variations of airline names. For instance, XXX and YYY both refer to the same airline but an exact match would miss that. To solve this issue, we can match two airline names that are almost similar. For instance, we can compute the edit distance between the two records and keep the records have a small distance (say at most 3). 

In [14]:
import py_stringsimjoin as ssj

# The py_stringsimjoin package requires the dataframe to have a column
# that can serve as a unique key. The next two lines add a column 'id'
# to each dataframe to address this.
clean_df['id'] = range(len(clean_df))
full_external_df['id'] = range(len(full_external_df))

# The inputs are:
# (1-2) left table and the right table, (3-4) the unique key in the left table and the right table,
# (5-6) the attribute to join on in the left table and the right table,
# (7-8) list of attributes to fetch from the left table and the right table,
# (9) the threshold for the edit distance; values with smaller edit distance would be considered a match.
similar_airlines = ssj.edit_distance_join(clean_df, full_external_df, 'id', 'id', 'Airline','Operator',
                                          l_out_attrs=['Airline', 'Date', 'Location', 'Flight'],
                                          r_out_attrs=['Operator', 'Flight #', 'Location', 'Fatalities'],
                                          threshold=3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


The above script only joins the airline name and does not take into account the flight numbers. To get the desired resutls, we can only select those rows that have matching flight numbers. Note that the join function appends the prefix 'r_' and 'l_' to the columns of the right and the left table respectively.

In [15]:
# getting those rows with matching flight numbers
combined = similar_airlines.loc[similar_airlines['r_Flight #'] == similar_airlines['l_Flight']]
print('We have found ' + str(len(combined)) + ' matching records!')
combined.sample(3)

We have found 41 matching records!


,_id,l_id,r_id,l_Airline,l_Date,l_Location,l_Flight,r_Operator,r_Flight #,r_Location,r_Fatalities,_sim_score
277,277,66,229,Caledonian Airways,"March 4, 1962,",Douala,153,Caledonian Airways,153,"Douala, Cameroon",111,0.0
137,137,19,78,United Airlines,"October 24, 1947","Utah, United States",608,United Air Lines,608,"Bryce Canyon, Utah",53,2.0
272,272,1,228,American Airlines,"January 14, 1936","Goodwin, Arkansas",1,American Airlines,1,"Jamaica Bay, New York, New York",95,0.0


This is an improvement over our previous results. We can take a look at rows in which the airline names don't exactly match to see the new matches that we have detected.

In [16]:
combined.loc[combined['l_Airline'] != combined['r_Operator'], ['l_Airline', 'r_Operator']]

,l_Airline,r_Operator
124,United Airlines,United Air Lines
137,United Airlines,United Air Lines
145,United Airlines,United Air Lines
168,United Airlines,United Air Lines
354,Japan Airlines,Japan Air Lines
420,United Airlines,United Air Lines
446,Japan Airlines,Japan Air Lines
516,Pacific Southwest Airlines,PacifiSouthwest Airlines
764,PMTair,PMT Air


#### Step 8) Linking the entities using other columns

What we have presented so far is by no means the best way we can approach the problem of linking entities. We only focused on a subset of columns to see what techniques can be applied.

An alternative approach to linking these entities would be to use the Date column in both datasets. This probably yields better results since (1) it's quite unlikely to have multiple incidents in a day, and (2) it's easier to check for an exact match. We need to consider the fact that Date format is different in the two datasets, but after cleaning the data, it might be the most useful column for matching the entities. 

**Task: Use the column 'Date' to find the matching between the two datasets. Note that attribute 'Date' is usually not missing, thus we can find more matches by considering all data points (even those with missing values in different columns).**

In [17]:
# >>>> to be done by students...

# use column 'Date' in df and external_df to find a matching incidents
# sub-step 1) Transform the Date in df to have the same format as the other dataset
# sub-step 2) Use dataframe's merge function to find an exact match
# sub-setp 3) Sample a few rows from the resulting table and see how many seems to be correct/incorrect